In [53]:
import pickle
import json
import string
import sys
import numpy as np
import pandas as pd
from numpy.linalg import norm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\theod\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\theod\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\theod\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [54]:
lem = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def lemmatize_sentence(sentence):
    tokens = word_tokenize(sentence)
    lemmatized_tokens = [lem.lemmatize(token) for token in tokens if token not in stop_words]
    return lemmatized_tokens
def load_pickle(path):
    with open(path, 'rb') as file:
        df =  pickle.load(file)
        df['sentence'] = df['sentence'].apply(lemmatize_sentence)
        return df
def load_json(path):
    with open(path, 'r') as file:
        return pd.read_json(file.read()).map(lem.lemmatize)
data20 = load_pickle('TrainingData/20news/df20.pkl')
datanyt = load_pickle('TrainingData/nyt/dfnyt.pkl')
seed20 = load_json('TrainingData/20news/seedwords.json')
seednyt = load_json('TrainingData/nyt/seedwords.json')

C:\Users\theod\AppData\Local\Temp\ipykernel_42304\3568182671.py:14: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(file.read()).map(lem.lemmatize)
C:\Users\theod\AppData\Local\Temp\ipykernel_42304\3568182671.py:14: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(file.read()).map(lem.lemmatize)


In [55]:
def train_model(data):
    model = Word2Vec(sentences=data['sentence'], vector_size=256, window=8, min_count=1, workers=16, ns_exponent = 1.1)

    # Save the trained model
    model.save("word2vec_model.bin")

    # You can also load the model later using:
    # model = Word2Vec.load("word2vec_model.bin")

    return model

In [56]:
model20 = train_model(data20)
modelnyt = train_model(datanyt)

In [65]:
def seed_vectors(seeds, model):
      return {col:sum([model.wv[seed] for seed in seeds[col]])/len(seeds[col]) for col in seeds}

In [66]:
seed_vecs20 = seed_vectors(seed20, model20)
seed_vecsnyt = seed_vectors(seednyt, modelnyt)

In [60]:
def doc_vectors(tokens, model):
    return [sum([model.wv[token] for token in doc])/len(doc) for doc in tokens]

In [61]:
doc_vecs20 = doc_vectors(data20['sentence'], model20)
doc_vecsnyt = doc_vectors(datanyt['sentence'], modelnyt)

In [62]:
def cos_sim(v1, v2):
    return np.dot(v1,v2)/(norm(v1)*norm(v2))

In [67]:
def compute_relevance(doc_vec, seed_vecs):
    out = []
    for doc in doc_vec:
        relevance = {seed_vec[0]:cos_sim(seed_vec[1], doc) for seed_vec in seed_vecs.items()}
        out.append(max(zip(relevance.values(), relevance.keys()))[1])
    return out

In [68]:
data20['pred_label'] = compute_relevance(doc_vecs20, seed_vecs20)
datanyt['pred_label'] = compute_relevance(doc_vecsnyt, seed_vecsnyt)

In [69]:
def score(expt, pred):
    return f1_score(expt, pred, average='macro'), f1_score(expt, pred, average='micro')

In [70]:
score(datanyt['label'], datanyt['pred_label']), score(data20['label'], data20['pred_label'])

((0.7275222438428319, 0.9273011191116509),
 (0.4882732333297365, 0.5373788268798948))

In [ ]:
data20

In [ ]:
datanyt['pred_label'].describe(), datanyt['label'].describe()

In [181]:
Word2Vec 0.92 0.83, 0.51 0.45

SyntaxError: invalid syntax (1065754375.py, line 1)

In [408]:
model2 = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, norm_only=True)

AttributeError: type object 'Word2Vec' has no attribute 'load_word2vec_format'

In [5]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [6]:
wv.save("google_model.bin")

In [ ]:
((0.7166912782367217, 0.8738613689598335)
 (0.4829792424322826, 0.5453201161071253))